# Real-time prediciton

A webcam feed that uses OpenCV and Keras to recognise gestures in real-time. Initialization involves loading a pre-trained model titled 'Gesture_CNN.h5' from a specified path. It uses a deque (from Python's collections module) to maintain a rolling window of the last 15 frames captured by the webcam, as the model appears to be based on 15-frame sequences. These 15 frames are processed by the predict_gesture function, which converts them into an appropriate shape and makes predictions using the imported model. The predicted gesture, which may be one of five pre-defined gestures such as 'Swiping Left' or 'Thumbs Up,' is then displayed on the current frame using the putText method of OpenCV. The live feed from the default camera is displayed in the 'Real-time Gesture Prediction' window. The capturing loop continues until the 'q' key is hit, after which the video capture is terminated and any OpenCV windows created are destroyed.


In [ ]:
import cv2
import numpy as np
from keras.models import load_model

# Load the trained model
model = load_model(r'C:\Users\kevin\Desktop\gesture recog\models\Gesture_CNN.h5')

# Using a queue to store the last 15 frames
from collections import deque
frames_queue = deque(maxlen=15)

def predict_gesture(frames_queue, model):
    # If we don't have 15 frames yet, return None
    if len(frames_queue) < 15:
        return None

    # Convert the queue to a numpy array
    frames_array = np.array(frames_queue)

    # Expand the dimensions to represent the batch size
    frames_array = np.expand_dims(frames_array, axis=0)  # Shape: (1, 15, 90, 90, 3)

    # Predict using the model
    predictions = model.predict(frames_array)

    # Convert predictions to desired output (e.g., gesture name)
    gesture_names = ['Swiping Left', 'Swiping Right', 'Stop', 'Thumps Down', 'Thumps Up']
    predicted_gesture = gesture_names[np.argmax(predictions)]

    return predicted_gesture

# Start capturing from the default camera
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize and add the frame to the frames queue
    frame_resized = cv2.resize(frame, (90, 90))
    frames_queue.append(frame_resized)

    # Get the gesture prediction for the current frame sequence
    gesture = predict_gesture(frames_queue, model)

    if gesture:
        # Display the prediction on the frame
        cv2.putText(frame, gesture, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show the frame
    cv2.imshow('Real-time Gesture Prediction', frame)

    # If 'q' key is pressed, exit loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
